In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer, mean_squared_error

warnings.filterwarnings("ignore")

In [2]:
game_data = pd.read_csv('../../generated_datasets/final_stats_df.csv')
odds_data = pd.read_csv('../../generated_datasets/final_odds_df.csv')

In [3]:
# Next step is pull home and away team data from the game_data dataframe at the latest date prior to the new game date

def get_latest_team_stats(team, date):
    team_games = game_data[(game_data['TEAM_ABBREVIATION_A'] == team) | (game_data['TEAM_ABBREVIATION_B'] == team)]
    try:
        latest_date = max(team_games[team_games['GAME_DATE'] < date]['GAME_DATE'])
    except: 
        return None
    latest_team_game = team_games[team_games['GAME_DATE'] == latest_date]
    if latest_team_game['TEAM_ABBREVIATION_A'].iloc[0] == team:
        return latest_team_game.iloc[:, np.r_[0, 4:21, -1]]
    elif latest_team_game['TEAM_ABBREVIATION_B'].iloc[0] == team:
        return latest_team_game.iloc[:, np.r_[0, 4:7, 21:36]]
    else:
        return None

In [4]:
# team_games = game_data[(game_data['TEAM_ABBREVIATION_A'] == 'CHI') | (game_data['TEAM_ABBREVIATION_B'] == 'CHI')]
# team_games[team_games['GAME_DATE'] == '2015-11-01']

In [5]:
# For every game in our odds data, pull the most up to date team stats for the home and away teams, such that the most recent date for the team stats occurs prior to the game date we are predicting

odds_date = []
odds_home = []
odds_away = []
odds_home_spread = []
odds_home_plus_minus = []

season_id = []
home_game_date = []
home_team_home_prior_ind = []
home_gp = []
home_sos = []
home_sos_l10 = []
home_win_pct = []
home_win_pct_l10 = []
home_3pt_pct = []
home_2pt_pct = []
home_pp100p = []
home_orb = []
home_drb = []
home_opp_3pt_pct = []
home_opp_2pt_pct = []
home_opp_pp100p = []

away_game_date = []
away_team_home_prior_ind = []
away_gp = []
away_sos = []
away_sos_l10 = []
away_win_pct = []
away_win_pct_l10 = []
away_3pt_pct = []
away_2pt_pct = []
away_pp100p = []
away_orb = []
away_drb = []
away_opp_3pt_pct = []
away_opp_2pt_pct = []
away_opp_pp100p = []

for i, game in odds_data.iterrows():
    try:
        # Pull home team's most recent stats
        home_stats = get_latest_team_stats(team=game['Home'], date=game['Date'])

        # Pull away team's most recent stats
        away_stats = get_latest_team_stats(team=game['Away'], date=game['Date'])

        if away_stats is not None and home_stats is not None:

            # Is it team A or team B from home_stats? Could be either one.
            try:
                home_stats['TEAM_A_SOS']
                team_letter = 'A'
                home_team_home_prior_ind.append(1)
            except:
                team_letter = 'B'
                home_team_home_prior_ind.append(0)

            season_id.append(home_stats['SEASON_ID'].item())
            home_game_date.append(home_stats['GAME_DATE'].item())
            home_gp.append(home_stats[f'TEAM_{team_letter}_GP'].item())
            home_sos.append(home_stats[f'TEAM_{team_letter}_SOS'].item())
            home_sos_l10.append(home_stats[f'TEAM_{team_letter}_SOS_LAST10'].item())
            home_win_pct.append(home_stats[f'TEAM_{team_letter}_WIN_PCT'].item())
            home_win_pct_l10.append(home_stats[f'TEAM_{team_letter}_WIN_PCT_LAST10'].item())
            home_3pt_pct.append(home_stats[f'TEAM_{team_letter}_3PT_PCT'].item())
            home_2pt_pct.append(home_stats[f'TEAM_{team_letter}_2PT_PCT'].item())
            home_pp100p.append(home_stats[f'TEAM_{team_letter}_PP100P'].item())
            home_orb.append(home_stats[f'TEAM_{team_letter}_ORB_PCT'].item())
            home_drb.append(home_stats[f'TEAM_{team_letter}_DRB_PCT'].item())
            home_opp_3pt_pct.append(home_stats[f'TEAM_{team_letter}_OPP_3PT_PCT'].item())
            home_opp_2pt_pct.append(home_stats[f'TEAM_{team_letter}_OPP_2PT_PCT'].item())
            home_opp_pp100p.append(home_stats[f'TEAM_{team_letter}_OPP_PP100P'].item())

            # Is it team A or team B from away_stats? Could be either one.
            try:
                away_stats['TEAM_A_SOS']
                team_letter = 'A'
                away_team_home_prior_ind.append(1)
            except:
                team_letter = 'B'
                away_team_home_prior_ind.append(0)

            away_game_date.append(away_stats['GAME_DATE'].item())
            away_gp.append(away_stats[f'TEAM_{team_letter}_GP'].item())
            away_sos.append(away_stats[f'TEAM_{team_letter}_SOS'].item())
            away_sos_l10.append(away_stats[f'TEAM_{team_letter}_SOS_LAST10'].item())
            away_win_pct.append(away_stats[f'TEAM_{team_letter}_WIN_PCT'].item())
            away_win_pct_l10.append(away_stats[f'TEAM_{team_letter}_WIN_PCT_LAST10'].item())
            away_3pt_pct.append(away_stats[f'TEAM_{team_letter}_3PT_PCT'].item())
            away_2pt_pct.append(away_stats[f'TEAM_{team_letter}_2PT_PCT'].item())
            away_pp100p.append(away_stats[f'TEAM_{team_letter}_PP100P'].item())
            away_orb.append(away_stats[f'TEAM_{team_letter}_ORB_PCT'].item())
            away_drb.append(away_stats[f'TEAM_{team_letter}_DRB_PCT'].item())
            away_opp_3pt_pct.append(away_stats[f'TEAM_{team_letter}_OPP_3PT_PCT'].item())
            away_opp_2pt_pct.append(away_stats[f'TEAM_{team_letter}_OPP_2PT_PCT'].item())
            away_opp_pp100p.append(away_stats[f'TEAM_{team_letter}_OPP_PP100P'].item())

            odds_date.append(odds_data.at[i, 'Date'])
            odds_home.append(odds_data.at[i, 'Home'])
            odds_away.append(odds_data.at[i, 'Away'])
            odds_home_spread.append(odds_data.at[i, 'Home Spread'])
            odds_home_plus_minus.append(odds_data.at[i, 'Home Team Plus Minus'])

        else: 
            pass

    except:
        # Append None or some other placeholder value to each list
        odds_date.append(None)
        odds_home.append(None)
        odds_away.append(None)
        odds_home_spread.append(None)
        odds_home_plus_minus.append(None)

        season_id.append(None)
        home_game_date.append(None)
        home_gp.append(None)
        home_team_home_prior_ind.append(None)
        home_sos.append(None)
        home_sos_l10.append(None)
        home_win_pct.append(None)
        home_win_pct_l10.append(None)
        home_3pt_pct.append(None)
        home_2pt_pct.append(None)
        home_pp100p.append(None)
        home_orb.append(None)
        home_drb.append(None)
        home_opp_3pt_pct.append(None)
        home_opp_2pt_pct.append(None)
        home_opp_pp100p.append(None)

        away_game_date.append(None)
        away_gp.append(None)
        away_team_home_prior_ind.append(None)
        away_sos.append(None)
        away_sos_l10.append(None)
        away_win_pct.append(None)
        away_win_pct_l10.append(None)
        away_3pt_pct.append(None)
        away_2pt_pct.append(None)
        away_pp100p.append(None)
        away_orb.append(None)
        away_drb.append(None)
        away_opp_3pt_pct.append(None)
        away_opp_2pt_pct.append(None)
        away_opp_pp100p.append(None)

In [6]:
# Construct dataframe for training
train_df = pd.DataFrame()

# Filtering variables
train_df['DATE'] = odds_date
train_df['DATE'] = pd.to_datetime(train_df['DATE'])
train_df['SEASON'] = [str(szn)[1:5] for szn in season_id]
train_df['HOME'] = odds_home
train_df['AWAY'] = odds_away
train_df['HOME_PRIOR_GAME_DATE'] = home_game_date
train_df['HOME_PRIOR_GAME_DATE'] = pd.to_datetime(train_df['HOME_PRIOR_GAME_DATE'])
train_df['HOME TEAM GP'] = home_gp
train_df['AWAY_PRIOR_GAME_DATE'] = away_game_date
train_df['AWAY_PRIOR_GAME_DATE'] = pd.to_datetime(train_df['AWAY_PRIOR_GAME_DATE'])
train_df['AWAY TEAM GP'] = away_gp

# Training variables
train_df['HOME SPREAD'] = odds_home_spread
train_df['HOME TEAM DAYS REST'] = (train_df['DATE'] - train_df['HOME_PRIOR_GAME_DATE']).dt.days - 1
train_df['HOME TEAM HOME PRIOR'] = home_team_home_prior_ind
train_df['HOME TEAM SOS'] = home_sos
train_df['HOME TEAM SOS LAST 10'] = home_sos_l10
train_df['HOME TEAM WIN PCT'] = home_win_pct
train_df['HOME TEAM WIN PCT LAST 10'] = home_win_pct_l10
train_df['HOME TEAM 3PT PCT'] = home_3pt_pct
train_df['HOME TEAM 2PT PCT'] = home_2pt_pct
train_df['HOME TEAM PP100P'] = home_pp100p
train_df['HOME TEAM ORB PCT'] = home_orb
train_df['HOME TEAM DRB PCT'] = home_drb
train_df['HOME TEAM OPP 3PT PCT'] = home_opp_3pt_pct
train_df['HOME TEAM OPP 2PT PCT'] = home_opp_2pt_pct
train_df['HOME TEAM OPP PP100P'] = home_opp_pp100p
train_df['AWAY TEAM DAYS REST'] = (train_df['DATE'] - train_df['AWAY_PRIOR_GAME_DATE']).dt.days - 1
train_df['AWAY TEAM HOME PRIOR'] = away_team_home_prior_ind
train_df['AWAY TEAM SOS'] = away_sos
train_df['AWAY TEAM SOS LAST 10'] = away_sos_l10
train_df['AWAY TEAM WIN PCT'] = away_win_pct
train_df['AWAY TEAM WIN PCT LAST 10'] = away_win_pct_l10
train_df['AWAY TEAM 3PT PCT'] = away_3pt_pct
train_df['AWAY TEAM 2PT PCT'] = away_2pt_pct
train_df['AWAY TEAM PP100P'] = away_pp100p
train_df['AWAY TEAM ORB PCT'] = away_orb
train_df['AWAY TEAM DRB PCT'] = away_drb
train_df['AWAY TEAM OPP 3PT PCT'] = away_opp_3pt_pct
train_df['AWAY TEAM OPP 2PT PCT'] = away_opp_2pt_pct
train_df['AWAY TEAM OPP PP100P'] = away_opp_pp100p

# Target variable
train_df['HOME PLUS MINUS'] = odds_home_plus_minus

# Get rid of games where teams have played < 10 games in a particular season
train_df = train_df[((train_df['HOME TEAM GP']>=10)&(train_df['AWAY TEAM GP']>=10))]

# Save and show
train_df.to_csv('../../generated_datasets/train_df.csv', index=False)
train_df.head()

,DATE,SEASON,HOME,AWAY,HOME_PRIOR_GAME_DATE,HOME TEAM GP,AWAY_PRIOR_GAME_DATE,AWAY TEAM GP,HOME SPREAD,HOME TEAM DAYS REST,...,AWAY TEAM WIN PCT LAST 10,AWAY TEAM 3PT PCT,AWAY TEAM 2PT PCT,AWAY TEAM PP100P,AWAY TEAM ORB PCT,AWAY TEAM DRB PCT,AWAY TEAM OPP 3PT PCT,AWAY TEAM OPP 2PT PCT,AWAY TEAM OPP PP100P,HOME PLUS MINUS
0,2023-04-09,2022,MIA,ORL,2023-04-07,81.0,2023-04-07,81.0,-5.0,1,...,0.5,0.345847,0.539955,108.212445,0.221682,0.717764,0.350301,0.566289,110.534516,13
1,2023-04-09,2022,CHI,DET,2023-04-07,81.0,2023-04-07,81.0,-8.5,1,...,0.1,0.352247,0.516595,107.154384,0.234513,0.670035,0.361954,0.568362,115.072100,22
2,2023-04-09,2022,TOR,MIL,2023-04-07,81.0,2023-04-07,81.0,-3.5,1,...,0.7,0.368276,0.557308,112.196853,0.234792,0.760560,0.354302,0.513276,108.373220,16
3,2023-04-09,2022,BOS,ATL,2023-04-07,81.0,2023-04-07,81.0,-4.5,1,...,0.6,0.353204,0.547396,113.005593,0.234015,0.706579,0.354120,0.562514,112.705425,6
4,2023-04-09,2022,BKN,PHI,2023-04-07,81.0,2023-04-07,81.0,-2.0,1,...,0.5,0.386303,0.550897,114.626792,0.202014,0.730845,0.347762,0.553923,109.655383,-29
